In [ ]:
 Kepler's 3 Laws

Law of Ellipses
Satellites don't move in perfect circles.  
→ They move in **oval (ellipse) orbits** around Earth.

 Law of Equal Areas
Satellites move **faster** when closer to Earth, and **slower** when farther.  
→ So, speed changes during orbit!

Law of Periods
Bigger orbits take more time.  
→ Time to complete one orbit depends on orbit size.

---

Why Use Kepler's Laws?

You can use them to:

Estimate satellite position** (X, Y, Z) at any time
Simulate satellite motion** when data is missing
Build physics-informed features** for your model (like velocity, distance, etc.)

# What Are Orbital Elements? (The 6 Satellite Numbers)

These are six values that describe a satellite's orbit. Think of them like the **"address and direction"** of a satellite's path around Earth:

| Orbital Element | What it Means |
|----------------|---------------|
| 1. **Semi-major axis (a)** | Size of the orbit (how big the ellipse is) |
| 2. **Eccentricity (e)** | How stretched the orbit is (0 = circle, closer to 1 = oval) |
| 3. **Inclination (i)** | Tilt of orbit compared to Earth's equator (in degrees) |
| 4. **RAAN (Ω)** | Where the orbit crosses the equator going north |
| 5. **Argument of Perigee (ω)** | Where in the orbit the satellite gets closest to Earth |
| 6. **Mean Anomaly (M)** | Where the satellite is currently in its orbit |

---

## How These 6 Values Help Us Get X, Y, Z

### Step 1: Use Kepler's Equation

We use the **Mean Anomaly (M)** and solve **Kepler's Equation**:

$$M = E - e \cdot \sin(E)$$

Here:
- **E** is **Eccentric Anomaly**, helps find the angle in the orbit
- We solve this using numerical methods like **Newton-Raphson**

### Step 2: Calculate True Anomaly (ν)

Once we get **E**, we find **True Anomaly**:

$$\nu = 2 \cdot \arctan\left(\sqrt{\frac{1+e}{1-e}} \cdot \tan\left(\frac{E}{2}\right)\right)$$

This tells us where the satellite is in the orbit.

### Step 3: Get Orbital Plane Coordinates

We use formulas to get **x_orbit** and **y_orbit** in the flat orbital plane:

$$r = \frac{a(1-e^2)}{1+e \cdot \cos(\nu)}$$

$$x_{orbital} = r \cdot \cos(\nu)$$

$$y_{orbital} = r \cdot \sin(\nu)$$

### Step 4: Rotate to 3D Earth Coordinates

Now we apply rotations using the other orbital elements:
- **Inclination (i)**
- **RAAN (Ω)**
- **Argument of Perigee (ω)**

We use **rotation matrices** to convert (x_orbital, y_orbital) into 3D (X, Y, Z) in Earth-centered space.

---

## Summary

**To get satellite X, Y, Z:**

1. Get the **6 orbital elements** (a, e, i, Ω, ω, M)
2. Use **Kepler's Law** to compute where the satellite is in its orbit
3. Convert that position from the orbit to **real 3D Earth space** using rotations

In [ ]:
from IPython.display import Markdown, display

md = """
🧠 **What Is a State-Space Model?**

A state-space model is a mathematical way to describe how something changes over time. In your case, it’s how satellite errors (X, Y, Z position and clock) change with time.

It has two main parts:

- **State Equation** → how the system (satellite errors) evolves from time t to t+1.
- **Observation Equation** → how you observe those errors with some noise.

Example:

At time t, you don’t know the real X_Error.  
But you can guess it based on how it changed before + measurements.

🔄 **What Is a Kalman Filter?**

The Kalman Filter is a smart algorithm that works with the state-space model. It predicts what will happen next, then corrects that guess using real data.

Two key steps:

- **Predict Step:** Uses math to guess the next error (e.g., X_Error at next timestamp).
- **Update Step:** Compares prediction with real measured data and corrects the guess.

It's like:

“I think the error will be 5 m.”  
“The real error is 4.6 m.”  
“Okay, let’s adjust my guess slightly.”
"""

display(Markdown(md))

# Kalman Filter Implementation for GNSS Error Forecasting

This notebook implements a Kalman filter pipeline to forecast satellite positioning errors for Day 8 based on training data from Days 1-7.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## Step 1: Load and Prepare Data

Load all training CSV files and combine them into a single dataset

In [ ]:
# Configuration
INPUT_FILES = [
    '../data/DATA_MEO_Train.csv',
    '../data/DATA_MEO_Train2.csv', 
    '../data/DATA_GEO_Train.csv'
]

# Load and combine all CSV files
def load_and_combine_data(file_list):
    """Load multiple CSV files and combine them"""
    dfs = []
    for file_path in file_list:
        try:
            df = pd.read_csv(file_path)
            # Add satellite type based on filename
            if 'MEO' in file_path:
                df['satellite_type'] = 'MEO'
            elif 'GEO' in file_path:
                df['satellite_type'] = 'GEO'
            dfs.append(df)
            print(f"✅ Loaded {file_path}: {len(df)} rows")
        except FileNotFoundError:
            print(f"⚠️ File not found: {file_path}")
    
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Load data
df_all = load_and_combine_data(INPUT_FILES)

# Parse timestamp column
df_all['timestamp'] = pd.to_datetime(df_all['utc_time'])

# Rename columns to standard names
df_all.rename(columns={
    'x_error (m)': 'X_Error',
    'y_error  (m)': 'Y_Error',
    'y_error (m)': 'Y_Error',
    'z_error (m)': 'Z_Error',
    'satclockerror (m)': 'Clock_Error'
}, inplace=True)

# Sort by timestamp
df_all.sort_values('timestamp', inplace=True)

print(f"\n📊 Total records: {len(df_all)}")
print(f"📅 Date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
print(f"\n🔍 Data preview:")
df_all.head(10)

## Step 2: Kalman Filter Implementation

Implement a 4D Kalman Filter for tracking X, Y, Z position errors and clock error

In [ ]:
class SimpleKalmanFilter:
    """
    Simple 4D Kalman Filter for GNSS error prediction
    State vector: [X_Error, Y_Error, Z_Error, Clock_Error]
    """
    
    def __init__(self, state_dim=4, meas_dim=4):
        self.dim_x = state_dim
        self.dim_z = meas_dim
        
        # State transition matrix (identity - assumes errors persist)
        self.F = np.eye(state_dim)
        
        # Measurement matrix (direct observation)
        self.H = np.eye(meas_dim)
        
        # Process noise covariance (how much we expect state to change)
        self.Q = np.diag([0.1, 0.1, 0.1, 10.0])
        
        # Measurement noise covariance (sensor uncertainty)
        self.R = np.diag([1.0, 1.0, 1.0, 100.0])
        
        # State estimate
        self.x = np.zeros((state_dim, 1))
        
        # State covariance (uncertainty in our estimate)
        self.P = np.diag([100.0, 100.0, 100.0, 1e4])
    
    def predict(self):
        """Predict next state"""
        # x = F * x
        self.x = self.F @ self.x
        
        # P = F * P * F' + Q
        self.P = self.F @ self.P @ self.F.T + self.Q
        
        return self.x.copy()
    
    def update(self, z):
        """Update state with measurement"""
        # Innovation (measurement residual)
        y = z - self.H @ self.x
        
        # Innovation covariance
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # Update state estimate
        self.x = self.x + K @ y
        
        # Update covariance
        I = np.eye(self.dim_x)
        self.P = (I - K @ self.H) @ self.P
        
        return self.x.copy()

print("✅ Kalman Filter class defined")

## Step 3: Train Kalman Filter on Days 1-7

Process training data to learn error patterns

In [ ]:
# Prepare data for training
target_cols = ['X_Error', 'Y_Error', 'Z_Error', 'Clock_Error']

# Remove rows with missing values in target columns
df_train = df_all.dropna(subset=target_cols).copy()

print(f"📊 Training samples: {len(df_train)}")
print(f"📅 Training period: {df_train['timestamp'].min()} to {df_train['timestamp'].max()}")

# Calculate data statistics for tuning filter parameters
for col in target_cols:
    std = df_train[col].std()
    mean = df_train[col].mean()
    print(f"{col:15s}: mean={mean:8.4f}, std={std:8.4f}")

# Initialize Kalman Filter
kf = SimpleKalmanFilter()

# Adjust measurement noise based on actual data variance
kf.R = np.diag([
    df_train['X_Error'].var(),
    df_train['Y_Error'].var(),
    df_train['Z_Error'].var(),
    df_train['Clock_Error'].var()
])

# Initialize with first measurement
first_obs = df_train.iloc[0][target_cols].values.reshape(-1, 1)
kf.x = first_obs.copy()

print("\n✅ Kalman Filter initialized with first observation")

In [ ]:
# Train Kalman Filter on historical data (Days 1-7)
filtered_states = []
predictions = []

print("🔄 Training Kalman Filter...")

for idx, row in df_train.iterrows():
    # Predict step
    pred = kf.predict()
    predictions.append(pred.flatten())
    
    # Update step with actual measurement
    z = row[target_cols].values.reshape(-1, 1)
    filtered = kf.update(z)
    filtered_states.append(filtered.flatten())

# Convert to arrays
filtered_states = np.array(filtered_states)
predictions = np.array(predictions)

print(f"✅ Processed {len(filtered_states)} observations")

# Calculate filter performance (how well it tracks known data)
errors = df_train[target_cols].values - filtered_states
rmse = np.sqrt(np.mean(errors**2, axis=0))

print("\n📈 Kalman Filter Performance (RMSE on training data):")
for i, col in enumerate(target_cols):
    print(f"  {col:15s}: {rmse[i]:.6f} m")

## Step 4: Forecast Day 8 Errors

Generate predictions for Day 8 at 15-minute intervals

In [ ]:
# Determine Day 8 start time
last_timestamp = df_train['timestamp'].max()
day8_start = (last_timestamp + pd.Timedelta(days=1)).normalize()

print(f"📅 Last training timestamp: {last_timestamp}")
print(f"📅 Day 8 starts: {day8_start}")

# Generate Day 8 timestamps (15-minute intervals for 24 hours)
DT_MINUTES = 15
num_intervals = int(24 * 60 / DT_MINUTES)  # 96 intervals

day8_timestamps = [day8_start + timedelta(minutes=DT_MINUTES * i) for i in range(num_intervals)]

print(f"\n🔮 Generating {num_intervals} predictions for Day 8...")

# Forecast Day 8 using only predict steps (no measurements available)
day8_predictions = []

for t in day8_timestamps:
    # Predict only (no update since we don't have measurements)
    pred = kf.predict()
    day8_predictions.append({
        'timestamp': t,
        'X_Error_pred': float(pred[0, 0]),
        'Y_Error_pred': float(pred[1, 0]),
        'Z_Error_pred': float(pred[2, 0]),
        'Clock_Error_pred': float(pred[3, 0])
    })

# Create DataFrame
df_day8_pred = pd.DataFrame(day8_predictions)

print(f"✅ Generated {len(df_day8_pred)} predictions")
print(f"\n🔍 First 5 predictions:")
df_day8_pred.head()

## Step 5: Save Predictions to CSV

In [ ]:
# Save predictions to CSV
OUTPUT_FILE = 'predicted_errors_day8.csv'

df_day8_pred.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Saved Day 8 predictions to: {OUTPUT_FILE}")
print(f"📊 Total predictions: {len(df_day8_pred)}")
print(f"\n📈 Prediction Summary:")
print(df_day8_pred.describe())

## Step 6: Visualize Results

Plot training data filtering and Day 8 forecasts

In [ ]:
# Visualize Kalman Filter performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🛰️ Kalman Filter: Training Performance & Day 8 Forecast', fontsize=16, fontweight='bold')

for idx, (col, ax) in enumerate(zip(target_cols, axes.flat)):
    # Plot training data (actual vs filtered)
    ax.plot(df_train['timestamp'], df_train[col], 'b.', alpha=0.3, label='Actual Measurements', markersize=3)
    ax.plot(df_train['timestamp'], filtered_states[:, idx], 'r-', linewidth=2, label='Kalman Filtered', alpha=0.8)
    
    # Plot Day 8 predictions
    ax.plot(df_day8_pred['timestamp'], df_day8_pred[f'{col}_pred'], 
            'g--', linewidth=2, label='Day 8 Forecast', marker='o', markersize=4)
    
    # Add vertical line at forecast boundary
    ax.axvline(x=day8_start, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Forecast Start')
    
    ax.set_xlabel('Time')
    ax.set_ylabel(f'{col} (m)')
    ax.set_title(col)
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## Summary

✅ **What We Did:**

1. **Loaded training data** from MEO and GEO satellite CSV files
2. **Implemented a 4D Kalman Filter** to track X, Y, Z position errors and clock error
3. **Trained the filter** on Days 1-7 historical data
4. **Generated forecasts** for Day 8 at 15-minute intervals (96 predictions)
5. **Saved predictions** to `predicted_errors_day8.csv`
6. **Visualized** the filter performance and forecasts

🎯 **Key Parameters:**
- State dimension: 4 (X, Y, Z, Clock errors)
- Forecast interval: 15 minutes
- Forecast horizon: 24 hours (Day 8)

📊 **Output:**
- CSV file with 96 predictions containing timestamps and forecasted errors
- Visualizations showing filter tracking and forecast trajectory

"""
Kalman filter pipeline for SIH GNSS error forecasting.
Reads SIH CSVs (MEO/GEO), runs a 4D Kalman filter per satellite on Day1-7,
and forecasts Day8 errors (15-minute intervals). Save predictions CSV.
"""

import pandas as pd
import numpy as np
from datetime import timedelta
import os

# ---------- CONFIG ----------
INPUT_FILES = ['../data/DATA_MEO_Train.csv', '../data/DATA_MEO_Train2.csv', '../data/DATA_GEO_Train.csv']
OUTPUT_PRED_CSV = 'predicted_errors_day8.csv'
DT_MINUTES = 15
STATE_DIM = 4  # [X_err, Y_err, Z_err, Clock_err]
MEAS_DIM = 4

# Initial covariances/noises (tune these)
P0_diag = np.array([100.0, 100.0, 100.0, 1e10])   # initial uncertainties
Q_diag = np.array([0.1, 0.1, 0.1, 10.0])          # process noise
R_diag = None  # compute from data below if possible

print("🚀 Starting Kalman Filter Pipeline...")
print("="*60)

In [ ]:
# Simple Kalman Filter implementation
class KalmanFilter:
    def __init__(self, dim_x=4, dim_z=4):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.F = np.eye(dim_x)  # State transition
        self.H = np.eye(dim_z)  # Measurement function
        self.Q = np.eye(dim_x)  # Process noise
        self.R = np.eye(dim_z)  # Measurement noise
        self.x = np.zeros((dim_x, 1))  # State
        self.P = np.eye(dim_x)  # Covariance
    
    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
    
    def update(self, z):
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(self.dim_x) - K @ self.H) @ self.P

print("✅ Kalman Filter class defined")

In [ ]:
# Helper functions
def load_and_concat(files):
    dfs = []
    for f in files:
        try:
            df = pd.read_csv(f)
            df['timestamp'] = pd.to_datetime(df['utc_time'])
            
            # Rename columns
            df.rename(columns={
                'x_error (m)': 'X_Error',
                'y_error  (m)': 'Y_Error',
                'y_error (m)': 'Y_Error',
                'z_error (m)': 'Z_Error',
                'satclockerror (m)': 'Clock_Error'
            }, inplace=True)
            
            # Add satellite ID based on file
            if 'MEO' in f:
                df['satellite_id'] = 'MEO'
            elif 'GEO' in f:
                df['satellite_id'] = 'GEO'
            
            dfs.append(df)
            print(f"✅ Loaded {f}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Error loading {f}: {e}")
    
    all_df = pd.concat(dfs, ignore_index=True)
    all_df.sort_values(['satellite_id', 'timestamp'], inplace=True)
    return all_df

def estimate_measurement_noise(df, target_cols):
    """Estimate measurement variance across entire data"""
    return np.diag([df[col].dropna().var() if df[col].dropna().shape[0]>0 else 1.0 for col in target_cols])

print("✅ Helper functions defined")

In [ ]:
# Main Pipeline - Load Data
df_all = load_and_concat(INPUT_FILES)
targets = ['X_Error','Y_Error','Z_Error','Clock_Error']

print(f"\n📊 Total records: {len(df_all)}")
print(f"📅 Date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
print(f"🛰️ Satellites: {df_all['satellite_id'].unique()}")

# Estimate R from data if available
R_est = estimate_measurement_noise(df_all, targets)
if R_diag is None:
    R_mat = R_est
else:
    R_mat = np.diag(R_diag)

print(f"\n📈 Measurement noise covariance (R):")
print(f"  X_Error: {R_mat[0,0]:.6f}")
print(f"  Y_Error: {R_mat[1,1]:.6f}")
print(f"  Z_Error: {R_mat[2,2]:.6f}")
print(f"  Clock_Error: {R_mat[3,3]:.6f}")

In [ ]:
# Build Day 8 timestamps
end_time = df_all['timestamp'].max()
day8_start = (end_time + pd.Timedelta(days=1)).normalize()  # midnight of next day
day8_times = [day8_start + pd.Timedelta(minutes=DT_MINUTES*i) for i in range(int(24*60/DT_MINUTES))]

print(f"\n🔮 Forecasting setup:")
print(f"  Last training time: {end_time}")
print(f"  Day 8 starts: {day8_start}")
print(f"  Forecast intervals: {len(day8_times)} ({DT_MINUTES} min each)")
print(f"  First forecast: {day8_times[0]}")
print(f"  Last forecast: {day8_times[-1]}")

In [ ]:
# Per-satellite Kalman filtering and forecasting
pred_rows = []

print("\n🔄 Processing satellites...")
print("="*60)

for sat, sat_df in df_all.groupby('satellite_id'):
    print(f"\n🛰️ Satellite: {sat}")
    
    sat_df = sat_df.set_index('timestamp').sort_index()
    
    # Initialize Kalman Filter
    kf = KalmanFilter(dim_x=STATE_DIM, dim_z=MEAS_DIM)
    kf.F = np.eye(STATE_DIM)
    kf.H = np.eye(MEAS_DIM)
    kf.Q = np.diag(Q_diag)
    kf.R = R_mat
    kf.x = np.zeros((STATE_DIM, 1))
    
    # Initialize state with first observation if exists
    first = sat_df.dropna(subset=targets).iloc[0] if sat_df.dropna(subset=targets).shape[0]>0 else None
    if first is not None:
        kf.x = np.array([[first['X_Error']],[first['Y_Error']],[first['Z_Error']],[first['Clock_Error']]])
        print(f"  Initial state: X={first['X_Error']:.3f}, Y={first['Y_Error']:.3f}, Z={first['Z_Error']:.3f}, Clk={first['Clock_Error']:.3f}")
    
    kf.P = np.diag(P0_diag)
    
    # Run KF over Day1-7 (predict + update)
    update_count = 0
    for t, row in sat_df.iterrows():
        # Predict
        kf.predict()
        
        # Update if measurement available
        if not any(np.isnan(row[targets].values)):
            z = np.array(row[targets]).reshape((MEAS_DIM, 1))
            kf.update(z)
            update_count += 1
    
    print(f"  Training updates: {update_count}")
    print(f"  Final state: X={kf.x[0,0]:.3f}, Y={kf.x[1,0]:.3f}, Z={kf.x[2,0]:.3f}, Clk={kf.x[3,0]:.3f}")
    
    # After Day7, forecast Day8: repeated predict steps
    for t in day8_times:
        # Predict only
        kf.predict()
        pred = kf.x.flatten()
        pred_rows.append({
            'timestamp': t,
            'satellite_id': sat,
            'X_Error_pred': float(pred[0]),
            'Y_Error_pred': float(pred[1]),
            'Z_Error_pred': float(pred[2]),
            'Clock_Error_pred': float(pred[3])
        })
    
    print(f"  Day 8 predictions: {len(day8_times)}")

print("\n✅ All satellites processed")

In [ ]:
# Save Day8 predictions
pred_df = pd.DataFrame(pred_rows)
pred_df.to_csv(OUTPUT_PRED_CSV, index=False)

print("\n" + "="*60)
print(f"💾 Saved Day 8 predictions to: {OUTPUT_PRED_CSV}")
print(f"📊 Total predictions: {len(pred_df)}")
print(f"🛰️ Satellites: {pred_df['satellite_id'].unique()}")
print(f"📅 Timestamp range: {pred_df['timestamp'].min()} to {pred_df['timestamp'].max()}")

print("\n📈 Prediction Statistics:")
for col in ['X_Error_pred', 'Y_Error_pred', 'Z_Error_pred', 'Clock_Error_pred']:
    print(f"  {col:20s}: mean={pred_df[col].mean():8.4f}, std={pred_df[col].std():8.4f}")

print("\n🔍 First 10 predictions:")
pred_df.head(10)

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🛰️ Day 8 GNSS Error Predictions (Kalman Filter)', fontsize=16, fontweight='bold')

error_cols = ['X_Error_pred', 'Y_Error_pred', 'Z_Error_pred', 'Clock_Error_pred']
titles = ['X Position Error', 'Y Position Error', 'Z Position Error', 'Clock Error']

for idx, (col, title, ax) in enumerate(zip(error_cols, titles, axes.flat)):
    for sat in pred_df['satellite_id'].unique():
        sat_data = pred_df[pred_df['satellite_id'] == sat]
        ax.plot(sat_data['timestamp'], sat_data[col], marker='o', markersize=3, 
                linewidth=2, label=f'{sat}', alpha=0.7)
    
    ax.set_xlabel('Time (Day 8)')
    ax.set_ylabel('Error (m)')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")